In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
# Tentukan direktori dataset
dataset_dir = './dataset'  # Sesuaikan path ini dengan path dataset Anda
img_height, img_width = 150, 150
batch_size = 32

# Membuat ImageDataGenerator untuk augmentasi data
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 19764 images belonging to 2 classes.
Found 4941 images belonging to 2 classes.


In [3]:
# Membangun model CNN dengan lebih banyak lapisan dan regularisasi
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Kompilasi model dengan learning rate scheduler
model.compile(optimizer=Adam(lr=1e-4), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Callback untuk mengatur learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [4]:
# Melatih model dengan callback
epochs = 50

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[reduce_lr, early_stop]
)

Epoch 1/50



617/617 [==============================] - 494s 795ms/step - loss: 0.8052 - accuracy: 0.7779 - val_loss: 0.4425 - val_accuracy: 0.8040 - lr: 0.0010
Epoch 2/50
617/617 [==============================] - 454s 735ms/step - loss: 0.4253 - accuracy: 0.8195 - val_loss: 0.3936 - val_accuracy: 0.8354 - lr: 0.0010
Epoch 3/50
617/617 [==============================] - 458s 742ms/step - loss: 0.4015 - accuracy: 0.8344 - val_loss: 0.5633 - val_accuracy: 0.6956 - lr: 0.0010
Epoch 4/50
617/617 [==============================] - 445s 721ms/step - loss: 0.3959 - accuracy: 0.8393 - val_loss: 0.5386 - val_accuracy: 0.7831 - lr: 0.0010
Epoch 5/50
617/617 [==============================] - 440s 713ms/step - loss: 0.3823 - accuracy: 0.8452 - val_loss: 0.3833 - val_accuracy: 0.8490 - lr: 0.0010
Epoch 6/50
617/617 [==============================] - 437s 708ms/step - loss: 0.3804 - accuracy: 0.8491 - val_loss: 0.5525 - val_accuracy: 0.7946 - lr: 0.0010
Epoch 7/50
617/617 [==============================] - 438

In [5]:
# Evaluasi model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

155/155 [==============================] - 44s 284ms/step - loss: 0.2407 - accuracy: 0.9093
Validation Accuracy: 90.93%


In [6]:
# import tensorflow as tf
# import numpy as np
# from tensorflow.keras.preprocessing import image

# # Fungsi untuk prediksi
# def predict_waste(image_path):
#     img = image.load_img(image_path, target_size=(150, 150))
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0) / 255.0
    
#     prediction = model.predict(img_array)
#     if prediction[0] > 0.5:
#         return 'Recyclable'
#     else:
#         return 'Organic'

# # Tes
# print(predict_waste('C:/Users/Adhi Pradityo/OneDrive/Documents/trashify/dataset/organic/organic_000077_photo.jpg'))


In [7]:
# Menyimpan model
model.save('trashify_model.h5')

c:\Users\Adhi Pradityo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
